In [68]:
from datetime import datetime
import glob
import os
import pandas as pd
import re

In [ ]:
daily_path = "daily.xlsx"
if os.path.exists(daily_path):
    os.remove(daily_path)
    print(f"{daily_path} has been deleted.")
else:
    print(f"{daily_path} not found.")

daily.xlsx has been deleted.


# Extract

In [70]:
# Read file
today = datetime.today().strftime(format='%Y-%m-%d')
file_path = glob.glob(f"DBA T-Systems JIRA {today}*.csv")[0]
df = pd.read_csv(file_path)

# Transform

In [71]:
df['Componente(s)'] = df['Componente(s)'].str.replace(r'^\w+-\d+-', '', regex=True)
df[['Componente(s)']].head()

,Componente(s)
0,Eina de QdC AUTOSERVEI
1,UNeix-(UNX)
2,Eina de QdC AUTOSERVEI
3,UNeix-(UNX)
4,UNeix-(UNX)


In [72]:
# Extreu WO/INC
df['Ticket'] = df['Resumen'].str.extract(r'\b(WO\d+|INC\d+)\b(?= - )', expand=False)
df['Resumen'] = df['Resumen'].str.replace(r'\s*(WO\d+|INC\d+)\s*-\s*', '', regex=True)
df[['Ticket', 'Resumen']].head()

,Ticket,Resumen
0,WO0000004544951,"Suport funcional de l'eina ""Plataforma Dades R..."
1,WO0000004542969,Error de longitud Fitxer 80 - IDIBELL per a AN...
2,WO0000004536702,"Suport tècnic per l'aplicació PDREU, del Depar..."
3,WO0000004534659,UNeix AM09 - Actualització CAR DT HIST DETALL ...
4,WO0000004367441,Suport funcional a UNeix AM09_23 . Problemes a...


In [73]:
# Reduir descripció
regex_pattern = r">>Descripció detallada:(.*?)>>Adreces de correu addicionals"
df["Descripción"] = df["Descripción"].str.extract(regex_pattern, flags=re.DOTALL)
df["Descripción"] = df["Descripción"].str.strip().str.replace("\n", "")
df[["Descripción"]].head()

,Descripción
0,"Molt bon dia, Voldríem sol·licitar una càrrega..."
1,La institució IDIBELL està carregant el fitxer...
2,"Bon dia, Aquest tiquet de suport tècnic és per..."
3,"Tiquet per l’aplicació UNEIX, destinat al lot ..."
4,NaN


In [74]:
# Renombra i ordena columnes
df = df.rename(columns={
    "Campo personalizado (Remedy Tiquet Status)": "Remedy Status",
    "Campo personalizado (Due Date Resolució ANS)": "ANS",
    "Campo personalizado (Linked Customer Code 5)": "Remedy ID",
    "Campo personalizado (Customer Reporter)": "Customer Reporter"
    })
df = df[['Componente(s)', 'Ticket', 'Resumen', 'Descripción', 'Estado', 'Remedy Status', 'Creada',
       'Resuelta', 'ANS', 'Remedy ID',
       'Customer Reporter']]
df.head(1)

,Componente(s),Ticket,Resumen,Descripción,Estado,Remedy Status,Creada,Resuelta,ANS,Remedy ID,Customer Reporter
0,Eina de QdC AUTOSERVEI,WO0000004544951,"Suport funcional de l'eina ""Plataforma Dades R...","Molt bon dia, Voldríem sol·licitar una càrrega...",Paralizada,Pending,17/04/2025 14:01,NaN,24/04/2025 15:49,REQ000011308263,XAVIER LASAUCA CISA


# Load

In [76]:
df.head(1)

,Componente(s),Ticket,Resumen,Descripción,Estado,Remedy Status,Creada,Resuelta,ANS,Remedy ID,Customer Reporter
0,Eina de QdC AUTOSERVEI,WO0000004544951,"Suport funcional de l'eina ""Plataforma Dades R...","Molt bon dia, Voldríem sol·licitar una càrrega...",Paralizada,Pending,17/04/2025 14:01,NaN,24/04/2025 15:49,REQ000011308263,XAVIER LASAUCA CISA


In [75]:
# Guardar Excel
df.to_excel(daily_path, index=False)
print(f"Excel '{daily_path}' guardat.")

Excel 'daily.xlsx' guardat.
